# Little Schemer Chapter 06

## The Function Quote

In [2]:
(quote a) ; same as 'a

a

In [3]:
(eq? (quote a) 'a)

#t

In [4]:
(quote +)

+

In [5]:
(eq? (quote +) '+)

#t

### Small Trick: How to get second element of a list?

In [6]:
(car (cdr '(hello OK 88))) ; this does the trick

OK

In [7]:
(car (cdr (cdr '(hello OK 888)))) ; this gets the 3rd element

888

## The Function Numbered?

The following are arithmetic expressions:

1. '3'
2. '3 + 4'
3. '3 + (4 * 5)'

Note in (3), the parenthesis are not part of the expression, they just mean the order of processing. So here is the function *numbered* which takes in an atom or a list, then tells whether that represents an arithmetic expression which consists of only numbers and operators (+ or \*)

In [8]:
(define numbered?
    (lambda (aexp)
        (cond
             ((atom? aexp) (number? aexp))
             ((or (eq? (car (cdr aexp)) (quote +)) (eq? (car (cdr aexp)) (quote *)))
              (and (numbered? (car aexp)) (numbered? (car (cdr (cdr aexp))))))
             (else #f)
        )
    )
)

In [9]:
(numbered? 3)

#t

In [10]:
(numbered? '(3 + 5))

#t

In [11]:
(numbered? '((3 + 5) * 4))

#t

In [12]:
(numbered? '((2 + 3) * (4 + 5)))

#t

## The Function Value

Compute the value of an arithmetic expression of the following form:

1. '3': input is a single number 3
2. 2 + 3: input is a list (2 + 3)
3. 2 + (3 * 5): input is a list (2 + (3 * 5))

The function takes a number or a list as input, where the list contains only numbers and + or * to form arithmetic expressions.

In [13]:
(define length
    (lambda (L)
        (cond
             ((null? L) 0)
             (else (+ 1 (length (cdr L))))
        )
    )
)

In [14]:
(define value
    (lambda (nexp)
        (cond
             ((atom? nexp) nexp)
             ((eq? (length nexp) 1) (value (car nexp)))
             ((eq? (car (cdr nexp)) (quote +)) (+ (value (car nexp)) (value (cdr (cdr nexp)))))
             ((eq? (car (cdr nexp)) (quote *)) (* (value (car nexp)) (value (cdr (cdr nexp)))))
        )
    )
)

In [15]:
(value 88)

88

In [16]:
(value '(2 * 3))

6

In [17]:
(value '(2 + (3 * 5)))

17

In [18]:
(value '(2 + ((2 + 5) * 4)))

30

### Can we simply it?

Actually we can. From the definition, we know that a valid input is either:

1. An atom, or
2. A list of 3 elements, with the second element being either + or *.

If it is case (2), then the first and third element can be either an atom of list. So here comes a simplified version of value, which does not need the 'length' function.

In [19]:
(define value2
    (lambda (nexp)
        (cond
             ((atom? nexp) nexp)
             ((eq? (car (cdr nexp)) (quote +)) (+ (value2 (car nexp)) (value2 (car (cdr (cdr nexp))))))
             ((eq? (car (cdr nexp)) (quote *)) (* (value2 (car nexp)) (value2 (car (cdr (cdr nexp))))))
        )
    )
)

In [20]:
(value2 99)

99

In [21]:
(value2 '(3 * 5))

15

In [22]:
(value2 '(2 + ((2 + 5) * 4)))

30

In [23]:
(value2 '(2 4))

#f

## Value Again

Let's say the arithmetic express are of the following form:

1. 3
2. (+ 3 4)
3. (* (3 + 4) 5)

Let's do the function value again.

In [24]:
(define value+
    (lambda (nexp)
        (cond
             ((atom? nexp) nexp)
             ((eq? (quote +) (car nexp)) (+ (value+ (car (cdr nexp))) (value+ (car (cdr (cdr nexp))))))
             ((eq? (quote *) (car nexp)) (* (value+ (car (cdr nexp))) (value+ (car (cdr (cdr nexp))))))
        )
    )
)

In [25]:
(value+ 3)

3

In [26]:
(value+ '(+ 3 4))

7

In [27]:
(value+ '(+ (+ 3 4) (* 5 7)))

42

### Question: What if an S-express is none of the above?

In [28]:
(value+ '(3 + 4)) ; Ooops, some nasty results, it gives a default value: False

#f

## Rewirte Value+ With Help Functions

We need to evaluate 3 things:

1. The operator: the first element 
2. The first operand (second element) and the second operand (third element).

In [35]:
(define operator
    (lambda (L)
        (car L)
    )
)


(define first-operand
    (lambda (L)
        (car (cdr L))
    )
)


(define second-operand
    (lambda (L)
        (car (cdr (cdr L)))
    )
)


(define value++
    (lambda (nexp)
        (cond
             ((atom? nexp) nexp)
             ((eq? (operator nexp) (quote +)) (+ (value++ (first-operand nexp)) (second-operand nexp)))
             ((eq? (operator nexp) (quote *)) (* (value++ (first-operand nexp)) (second-operand nexp)))
        )
    )
)

In [36]:
(value++ 3)

3

In [37]:
(value++ '(+ 3 5))

8

In [38]:
(value++ '(+ (* 2 4) 9))

17